<a href="https://colab.research.google.com/github/VivianYueh/Imformation-Retrieval-Final/blob/main/%E9%97%9C%E9%8D%B5%E5%AD%97%E6%93%B7%E5%8F%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>此處是使用處理中文句子斷詞的範例，包括CKIPtagger及Distiltag</h3>

In [ ]:
!pip install -U DistilTag

  Using cached DistilTag-0.2.2-py3-none-any.whl.metadata (390 bytes)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
import DistilTag
DistilTag.download()

Downloading...
From (original): https://drive.google.com/uc?id=1AzUICPQ5MMt_IWg4JZ3mWM6vGbQkv01L
From (redirected): https://drive.google.com/uc?id=1AzUICPQ5MMt_IWg4JZ3mWM6vGbQkv01L&confirm=t&uuid=844a4526-217c-47f9-b9f9-dd92981e6851
To: /tmp/tmpsam65fw6distiltag/tagmodel.zip
100%|██████████| 501M/501M [00:10<00:00, 46.6MB/s]


setting up model...
DistilTag model installed.


In [ ]:
from DistilTag import DistilTag
tagger = DistilTag()

### 保留名詞、動詞

In [ ]:
def filter_nouns_verbs(tagged):
    """
    過濾出名詞(N*)與動詞(V*)的詞性標註
    """
    return [(word, pos) for word, pos in tagged if pos.startswith('N') or pos.startswith('V')]


### 擷取關鍵字

In [ ]:
def unique_words(words):
    return list(dict.fromkeys(words))  # 去重同時保留原順序

In [ ]:
import requests
from bs4 import BeautifulSoup

base_url = 'https://www.niar.org.tw'
news_base_url = 'https://www.niar.org.tw/xmdoc?xsmsid=0I148622737263495777'

page = 1
news_count = 0

print("國家實驗研究院新聞列表：\n")

while news_count<10:
    url = f"{news_base_url}&page={page}"
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    rows = soup.find_all('tr')

    found_news = False

    for row in rows:
        date_td = row.find('td', class_='date')
        title_td = row.find('td', class_='title')

        if date_td and title_td:
            found_news = True
            news_count += 1
            date = date_td.text.strip()
            a_tag = title_td.find('a')
            title = a_tag.text.strip()
            tags = tagger.tag(title)  # 這裡可能是 list[list[tuple]]
            filtered = [filter_nouns_verbs(sent) for sent in tags]
            words = [word for sent in filtered for word, pos in sent]
            unique = unique_words(words)
            link = base_url + a_tag['href']
            print(f"{news_count}. [{date}] {title}\n   {tags}\n   {unique}\n   👉 {link}\n")

    if not found_news:
        break  # 沒有新內容就停止
    page += 1

國家實驗研究院新聞列表：

1. [2025-04-16] 科教館「科學家的秘密基地」 4/16更新展區重新亮相
   [[('科教館', 'Nc'), ('「', 'PARENTHESISCATEGORY'), ('科學家', 'Na'), ('的', 'DE'), ('秘密', 'Na'), ('基地', 'Nc'), ('」', 'PARENTHESISCATEGORY'), ('４／１６', 'Nd'), ('更新', 'VC'), ('展區', 'Nc'), ('重新', 'D'), ('亮相', 'VA')]]
   ['科教館', '科學家', '秘密', '基地', '４／１６', '更新', '展區', '亮相']
   👉 https://www.niar.org.tw/xmdoc/cont?xsmsid=0I148622737263495777&sid=0P106529706322578269

2. [2025-04-14] 【國研院說明稿】關於小油坑失火事件  若鑑定結果為國研院設備造成 國研院絕不卸責
   [[('【', 'PARENTHESISCATEGORY'), ('國研院', 'Nc'), ('說明稿', 'Na'), ('】', 'PARENTHESISCATEGORY'), ('關於', 'P'), ('小油坑', 'Nc'), ('失火', 'VH'), ('事件', 'Na'), ('若', 'Cbb'), ('鑑定', 'VC'), ('結果', 'Na'), ('為', 'P'), ('國研院', 'Nc'), ('設備', 'Na'), ('造成', 'VG'), ('國研院', 'Nc'), ('絕不', 'D'), ('卸責', 'VA')]]
   ['國研院', '說明稿', '小油坑', '失火', '事件', '鑑定', '結果', '設備', '造成', '卸責']
   👉 https://www.niar.org.tw/xmdoc/cont?xsmsid=0I148622737263495777&sid=0P104744411632087760

3. [2025-04-14] 【國研院說明稿】關於小油坑失火事件
   [[('【', 'PARENTHESISCATEGORY'), ('國研院',

### 內文擷取

In [34]:
import requests
from bs4 import BeautifulSoup
import textwrap
import json
from datetime import datetime
import os
from google.colab import files


dict_list = []
latest_saved_date = None

json_file = "content_data.json"
dict_list = []
latest_saved_date = None

if os.path.exists(json_file):
    with open(json_file, "r", encoding="utf-8") as f:
        try:
            dict_list = json.load(f)
            if dict_list:
                latest_saved_date = datetime.strptime(dict_list[0]["date"], "%Y-%m-%d")
                print(f"  已找到 {json_file}，最新日期為：{latest_saved_date.date()}")
        except Exception as e:
            print(f"  讀取 JSON 發生錯誤：{e}，將從頭開始抓取")
else:
    print(f"  未發現 {json_file}，將從頭開始抓資料（最多 {200} 筆）")


base_url = 'https://www.niar.org.tw'
news_base_url = 'https://www.niar.org.tw/xmdoc?xsmsid=0I148622737263495777'

page = 1
news_count = 0
new_items = []
stop=False

#print("國家實驗研究院新聞列表：\n")

while not stop:
    url = f"{news_base_url}&page={page}"
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    rows = soup.find_all('tr')

    found_news = False

    for row in rows:
        date_td = row.find('td', class_='date')
        title_td = row.find('td', class_='title')

        if date_td and title_td:
            found_news = True
            news_count += 1
            date = date_td.text.strip()

            try:
              date_obj = datetime.strptime(date, "%Y-%m-%d")
            except ValueError:
              continue  # 跳過無法解析的日期

            # 判斷是否是舊資料（小於或等於）
            if latest_saved_date and date_obj <= latest_saved_date:
              stop = True
              break

            a_tag = title_td.find('a')
            title = a_tag.text.strip()
            link = base_url + a_tag['href']
            article_response = requests.get(link)
            article_response.encoding = 'utf-8'
            article_soup = BeautifulSoup(article_response.text, 'html.parser')

            # 抓取文章內容
            article_response = requests.get(link)
            article_response.encoding = 'utf-8'
            article_soup = BeautifulSoup(article_response.text, 'html.parser')
            content = article_soup.find_all('p', class_='MsoNormal')
            content_text = "\n".join(p.get_text(strip=True)  for p in content  if p.get_text(strip=True))
            #print(f"{news_count}. {title}\n   {content_text}\n")
            #print(f"{news_count}. [{date}] {title}\n   {tags}\n   {unique}\n   👉 {link}\n")

            content_dict={"title":title,"date":date,"content":content_text,"link":link}
            new_items.append(content_dict)

            if not latest_saved_date and len(new_items) >= 200:
              stop = True
              break

    print(f"\n  共新增 {len(new_items)} 筆新聞資料")

    if not found_news:
        break  # 沒有新內容就停止
    page += 1


if new_items:
  dict_list = new_items + dict_list
  print(f"\n  共新增 {len(new_items)} 筆新聞資料")
  with open("content_data.json", "w", encoding="utf-8") as f:
    json.dump(dict_list, f, ensure_ascii=False, indent=2)
files.download("content_data.json")

  未發現 content_data.json，將從頭開始抓資料（最多 200 筆）

  共新增 10 筆新聞資料

  共新增 20 筆新聞資料

  共新增 30 筆新聞資料

  共新增 40 筆新聞資料

  共新增 50 筆新聞資料

  共新增 60 筆新聞資料

  共新增 70 筆新聞資料

  共新增 80 筆新聞資料

  共新增 90 筆新聞資料

  共新增 100 筆新聞資料

  共新增 110 筆新聞資料

  共新增 120 筆新聞資料

  共新增 130 筆新聞資料

  共新增 140 筆新聞資料

  共新增 150 筆新聞資料

  共新增 160 筆新聞資料

  共新增 170 筆新聞資料

  共新增 180 筆新聞資料

  共新增 190 筆新聞資料

  共新增 200 筆新聞資料

  共新增 200 筆新聞資料


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 4/17

In [ ]:
import requests
from bs4 import BeautifulSoup
import textwrap
from collections import Counter

base_url = 'https://www.niar.org.tw'
news_base_url = 'https://www.niar.org.tw/xmdoc?xsmsid=0I148622737263495777'

page = 1
news_count = 0

special_terms = [
    "國家生物模式中心",
    "國家地震工程研究中心",
    "國家高速網路與計算中心",
    "台灣半導體研究中心",
    "國家儀器科技研究中心",
    "科技政策研究與資訊中心",
    "台灣海洋科技研究中心"
]

print("國家實驗研究院新聞列表：\n")

while news_count<10:
    url = f"{news_base_url}&page={page}"
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    rows = soup.find_all('tr')

    found_news = False

    for row in rows:
        date_td = row.find('td', class_='date')
        title_td = row.find('td', class_='title')

        if date_td and title_td:
            found_news = True
            news_count += 1
            #date = date_td.text.strip()
            a_tag = title_td.find('a')
            title = a_tag.text.strip()
            '''tags = tagger.tag(title)  # 這裡可能是 list[list[tuple]]
            filtered = [filter_nouns_verbs(sent) for sent in tags]
            words = [word for sent in filtered for word, pos in sent]
            unique = unique_words(words)'''
            link = base_url + a_tag['href']
            article_response = requests.get(link)
            article_response.encoding = 'utf-8'
            article_soup = BeautifulSoup(article_response.text, 'html.parser')

            # 抓取文章內容
            article_response = requests.get(link)
            article_response.encoding = 'utf-8'
            article_soup = BeautifulSoup(article_response.text, 'html.parser')
            content = article_soup.find_all('p', class_='MsoNormal')
            content_text = "\n".join(p.get_text(strip=True)  for p in content  if p.get_text(strip=True))

            found_special_terms = [term for term in special_terms if term in content_text]

            tags = tagger.tag(content_text)
            filtered = [filter_nouns_verbs(sent) for sent in tags]
            words = [word for sent in filtered for word, pos in sent]

            for term in found_special_terms:
              for subword in term:
                  words = [word for word in words if word != subword]

            words += found_special_terms

            counter = Counter(words)
            top_10 = counter.most_common(10)
            print(f"{news_count}. {title}\n   {top_10}\n   {found_special_terms}\n")

            #print(f"{news_count}. {title}\n   {content_text}\n")
            #print(f"{news_count}. [{date}] {title}\n   {tags}\n   {unique}\n   👉 {link}\n")

    if not found_news:
        break  # 沒有新內容就停止
    page += 1


國家實驗研究院新聞列表：

1. 科教館「科學家的秘密基地」 4/16更新展區重新亮相
   [('中心', 17), ('海洋', 13), ('基地', 12), ('研究', 8), ('國家', 7), ('動物', 7), ('實驗', 6), ('參觀', 6), ('基因', 6), ('者', 6)]
   ['國家生物模式中心', '國家地震工程研究中心', '台灣海洋科技研究中心']

2. 【國研院說明稿】關於小油坑失火事件  若鑑定結果為國研院設備造成 國研院絕不卸責
   [('國研院', 8), ('中心', 5), ('陽明山', 3), ('國家', 3), ('表示', 3), ('國網', 3), ('設備', 3), ('調查', 3), ('空氣', 3), ('資料', 3)]
   ['國家高速網路與計算中心']

3. 【國研院說明稿】關於小油坑失火事件
   [('空氣', 6), ('品質', 6), ('感測器', 5), ('國家', 4), ('地區', 3), ('中心', 2), ('小油坑', 2), ('陽明山', 2), ('公園', 2), ('本', 2)]
   ['國家高速網路與計算中心']

4. 臺法電子資訊與器官晶片深化交流 合作提升雙方科技研發量能
   [('研究', 12), ('法國', 11), ('技術', 10), ('器官', 9), ('國家', 8), ('國研院', 7), ('合作', 7), ('晶片', 7), ('研究院', 5), ('交流', 5)]
   ['國家生物模式中心', '國家儀器科技研究中心']

5. 國立陽明交通大學、振興醫院與國研院國儀中心 共同開發新型微米級光纖麥克風 為聽障者提供更無礙的聲音感受
   [('麥克風', 13), ('光纖', 10), ('聲音', 7), ('中心', 7), ('國儀', 6), ('應用', 5), ('薄膜', 5), ('此', 4), ('新型', 4), ('微米級', 4)]
   ['國家儀器科技研究中心']

6. TAIWAN AI RAP試營運啟動 徵求試營運用戶，免費算力助攻AI開發！
   [('開發', 19), ('應用', 15), ('中心', 7), ('服務

## ckiptagger

In [3]:
!pip install -U ckip-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

In [41]:
# Initialize drivers
ws_driver  = CkipWordSegmenter(model="bert-base")
pos_driver = CkipPosTagger(model="bert-base")
ner_driver = CkipNerChunker(model="bert-base")

In [42]:
existing_content=[]
if os.path.exists("content_data.json"):
    with open("content_data.json", "r", encoding="utf-8") as f:
        try:
            dict_list = json.load(f)
            existing_content = {dict_list[i]["content"] for i in range(0,5)}
        except Exception as e:
            print("讀取舊資料失敗")

In [43]:
ws  = ws_driver(existing_content)
pos = pos_driver(ws)
ner = ner_driver(existing_content)

Inference: 100%|██████████| 1/1 [00:18<00:00, 18.25s/it]


In [44]:
# Pack word segmentation and part-of-speech results
def pack_ws_pos_sentece(sentence_ws, sentence_pos):
   assert len(sentence_ws) == len(sentence_pos)
   res = []
   for word_ws, word_pos in zip(sentence_ws, sentence_pos):
      res.append(f"{word_ws}({word_pos})")
   return "\u3000".join(res)

# Show results
for sentence, sentence_ws, sentence_pos, sentence_ner in zip(existing_content, ws, pos, ner):
   print(sentence)
   print(pack_ws_pos_sentece(sentence_ws, sentence_pos))
   for entity in sentence_ner:
      print(entity)
   print()

國科會轄下之國家實驗研究院及國家太空中心，與國立臺灣科學教育館合作，辦理「科學家的秘密基地」長期展，自2023年3月開展以來，備受各界好評，已吸引超過13萬人入場參觀。今（4/16）日於更新部分展品後重新開展，希望能幫助參觀民眾透過互動遊戲、親眼觀察與模型展示，習得有趣的科學知識，同時深入認識科研工作。「科學家的秘密基地」位於科教館8樓東南側扇形展場，分為「實驗基地」、「探測基地」和「智慧基地」三區，每一區均包含兩大展示主題。此次更新的是國家生物模式中心（原國家實驗動物中心）、國家地震工程研究中心（國震中心）及台灣海洋科技研究中心（海洋中心）的展品。一進門就看到實驗基地－生物模式中心展出的「貓咪毛色變化的秘密」，這裡展示多基因協同作用的觀念。生物模式中心近年以革新的基因編輯工具「CRISPR/Cas」基因剪刀產製「擬人鼠」動物模式，推進我國精準醫療研究與臨床應用，而科學家進行基因編輯時，須嚴加考慮生物體複雜的基因層次。現場除了深入淺出的科學觀念，也設計了一個基因密碼轉盤遊戲，參觀者可以透過簡單的科普工具，輕鬆學習生物醫學知識。實驗基地－國震中心透過互動展示，介紹房屋的共振原理，包括不同樓高建築物的振動特性，以及地震波在不同地質條件下傳遞所產生的共振效應。現場提供以不同長度的吸管模擬各種樓高的共振教具，讓參觀者能親身體驗結構與地震波共振的現象。同時展示「滾動式隔震平台」，結合電動振動台模擬真實地震波，可清楚觀察放置於隔震裝置上的設備，其振動幅度大幅減少，讓參觀者直觀感受隔震技術的保護效果。接著來到探測基地－海洋中心，這裡展出於臺灣西南海域及南海採集到的海洋浮游動物標本、寫真、採集工具，以及使用海洋中心自製底碇平台記錄之水下300米的海底世界。海洋浮游動物在海洋生態系中扮演著重要而多樣的角色，牠們的種類及數量非常容易受到海洋環境的影響，當環境變動時（如溫度、鹽度、酸鹼度），浮游動物的組成也會隨之改變，間接對魚類、鯨類等各種海洋動物產生連鎖反應。因此浮游動物是一種很好的環境指標生物，可以用來監測海洋環境變遷、預測海洋生態系的改變。展場設置顯微觀察區，讓參觀者一窺這群嬌小但至關重要的小小兵身影，了解牠們是維護海洋生態平衡的關鍵多數。除了更新的三件展品外，探測基地－太空中心展出我國第一顆自製氣象衛星獵風者衛星（Triton）的1：1模型及其元件介紹。獵風者衛星搭載國家太空中心

### 關鍵詞提取

In [14]:
!pip install keybert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.7 MB/s eta 0:00:00


In [15]:
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer

In [24]:
def ws_zh(text):
    words = ws_driver([text])
    return words[0]

In [25]:
vectorizer = CountVectorizer(tokenizer=ws_zh)

In [49]:
kw_model = KeyBERT()
#kw_model = KeyBERT(model='distiluse-base-multilingual-cased-v1')

In [50]:
for i in range(0,5):

  keywords = kw_model.extract_keywords(dict_list[i]["content"],vectorizer=vectorizer)
  print(dict_list[i]["content"])
  print(keywords)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Inference: 100%|██████████| 1/1 [00:05<00:00,  5.13s/it]
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


國科會轄下之國家實驗研究院及國家太空中心，與國立臺灣科學教育館合作，辦理「科學家的秘密基地」長期展，自2023年3月開展以來，備受各界好評，已吸引超過13萬人入場參觀。今（4/16）日於更新部分展品後重新開展，希望能幫助參觀民眾透過互動遊戲、親眼觀察與模型展示，習得有趣的科學知識，同時深入認識科研工作。「科學家的秘密基地」位於科教館8樓東南側扇形展場，分為「實驗基地」、「探測基地」和「智慧基地」三區，每一區均包含兩大展示主題。此次更新的是國家生物模式中心（原國家實驗動物中心）、國家地震工程研究中心（國震中心）及台灣海洋科技研究中心（海洋中心）的展品。一進門就看到實驗基地－生物模式中心展出的「貓咪毛色變化的秘密」，這裡展示多基因協同作用的觀念。生物模式中心近年以革新的基因編輯工具「CRISPR/Cas」基因剪刀產製「擬人鼠」動物模式，推進我國精準醫療研究與臨床應用，而科學家進行基因編輯時，須嚴加考慮生物體複雜的基因層次。現場除了深入淺出的科學觀念，也設計了一個基因密碼轉盤遊戲，參觀者可以透過簡單的科普工具，輕鬆學習生物醫學知識。實驗基地－國震中心透過互動展示，介紹房屋的共振原理，包括不同樓高建築物的振動特性，以及地震波在不同地質條件下傳遞所產生的共振效應。現場提供以不同長度的吸管模擬各種樓高的共振教具，讓參觀者能親身體驗結構與地震波共振的現象。同時展示「滾動式隔震平台」，結合電動振動台模擬真實地震波，可清楚觀察放置於隔震裝置上的設備，其振動幅度大幅減少，讓參觀者直觀感受隔震技術的保護效果。接著來到探測基地－海洋中心，這裡展出於臺灣西南海域及南海採集到的海洋浮游動物標本、寫真、採集工具，以及使用海洋中心自製底碇平台記錄之水下300米的海底世界。海洋浮游動物在海洋生態系中扮演著重要而多樣的角色，牠們的種類及數量非常容易受到海洋環境的影響，當環境變動時（如溫度、鹽度、酸鹼度），浮游動物的組成也會隨之改變，間接對魚類、鯨類等各種海洋動物產生連鎖反應。因此浮游動物是一種很好的環境指標生物，可以用來監測海洋環境變遷、預測海洋生態系的改變。展場設置顯微觀察區，讓參觀者一窺這群嬌小但至關重要的小小兵身影，了解牠們是維護海洋生態平衡的關鍵多數。除了更新的三件展品外，探測基地－太空中心展出我國第一顆自製氣象衛星獵風者衛星（Triton）的1：1模型及其元件介紹。獵風者衛星搭載國家太空中心

Inference: 100%|██████████| 1/1 [00:01<00:00,  1.33s/it]
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


對於小油坑失火事件，陽明山國家公園管理處表示，初步研判此次火災有可能是國家實驗研究院國家高速網路與計算中心（國研院國網中心）之設備造成，國研院院長蔡宏營對此表示，國研院將慎重面對，如果最後調查結果確實是國研院國網中心之設備造成，國研院將承擔相關責任，絕不卸責。國研院已啟動內部調查，並將全力配合消防單位之外部調查，釐清起火原因；同時亦將仔細盤點現有設備，預防類似事件再度發生。國研院表示，國研院國網中心在小油坑地區安裝空氣品質感測器，並透過用於防災的無線電波段Band 20，將感測資料傳輸回國網中心主機，驗證Band 20在沒有商用無線網路5G、6G及惡劣環境中，亦可傳輸資料，對於提升防救災工作的通訊傳輸能力有很大幫助；同時藉由空氣品質感測器，收集陽明山區域過去1年8個月的PM 2.5資料，在民生公共物聯網上對外公開，提供各界使用，幫助學研界增加對陽明山地區空氣污染情形的了解。
[('起火', 0.3694), ('失火', 0.3694), ('中心', 0.3566), ('火災', 0.3405), ('能力', 0.2752)]


Inference: 100%|██████████| 1/1 [00:01<00:00,  1.29s/it]
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


為偵測大屯火山活動噴發氣體與空氣品質，國家實驗研究院國家高速網路與計算中心（國研院國網中心）利用空氣品質感測器，在小油坑地區蒐集大屯山地區的空氣品質資料。
小油坑地區位於陽明山國家公園管理範圍內，本計畫於2022年正式向陽明山國家公園管理處提出申請，並獲得同意，程序合法合規，且已於2024年底結束。
本計畫係委託廠商架設空氣品質感測器，透過網路線即時回傳空品數據。電力來源是透過太陽能供電系統，保障在偏遠無電區域仍可持續運作。
承包廠商以太陽能桿提供空氣品質感測器所需電力，下圖即為太陽能板發電與儲電設備及空氣品質感測器。太陽能桿與空品感測器皆為合法合規之設備，並經測試與驗證後才進行部署。
[('中心', 0.3396), ('空氣', 0.333), ('空品', 0.333), ('電力', 0.2869), ('保障', 0.2785)]


Inference: 100%|██████████| 1/1 [00:05<00:00,  5.43s/it]
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


國科會轄下之國家實驗研究院（國研院）與法國原子能暨替代能源總署（CEA）轄下之電子暨資訊技術研究室（CEA-Leti），於臺灣時間4月10日簽訂合作備忘錄，未來將以雙邊工作坊深化技術交流。法國國家健康與醫學研究院（Inserm）則與國研院於4月7日在巴黎辦理「臺法雙邊器官晶片科學論壇」，期能結合法國在基礎醫學實驗與我國在半導體、儀器技術的專長，建立實質的合作研究計畫，並將我國的生物晶片技術推展至歐洲。法國電子暨資訊技術研究室是法國最重要的電子資訊研究單位，專精於微電子和奈米技術應用。國研院與法國電子暨資訊技術研究室未來將在半導體、生醫、光學技術等領域共同舉辦工作坊或研討會，並進行人才交流互訪，期能促進雙方實質技術交流與研究合作。法國國家健康與醫學研究院則是法國首屈一指的生醫研究單位，此次與國研院邀集臺法雙方各12個具藥物開發應用潛力、且已完成概念性驗證或雛型品開發之研究團隊，共同辦理「臺法雙邊器官晶片科學論壇」，就生物材料（Biomaterial）、感測器與影像（Sensor and Imaging）及多功能器官晶片（Multicomponent & Multifunctional OoC）三大主題交流討論，期望透過成果媒合，促成合作研究的機會，加成雙方研發優勢。器官晶片是國科會與法國高等教育暨研究部於2024年臺法科學研究會議中列出的合作項目之一，是全球最受矚目的活體動物替代方法，結合了3D立體細胞培養、微流道、生醫感測器及人工智慧等技術，在實驗室重建器官內部的結構與環境。若模擬人體健康器官功能進行試藥，可用於藥物毒性或副作用的測試；若模擬生病的器官，則有助於藥物功效的測試與篩選；若能建立病人的器官晶片，更有機會直接運用於個人化醫療策略的輔助評估。此次參與論壇的臺灣團隊來自國科會生科處專案補助之「動物實驗替代科技研發計畫」，以及國研院的「器官晶片多元驗證平台」，包括陽明交通大學、清華大學、臺北醫學大學、國家衛生研究院、長庚醫院，以及國研院國家生物模式中心（原國家實驗動物中心）與國家儀器科技研究中心（原台灣儀器科技研究中心）的研究團隊。論壇結束後，國研院蔡宏營院長並率隊參訪法國巴黎－薩克雷大學（UPSaclay）及法國國家科學研究院（CNRS），期能建立臺法雙方更緊密、更全面的合作關係，藉由實習生交流、大型研究設施共享、合作執行研究計畫等，提升臺法雙方在生醫、高速計

Inference: 100%|██████████| 1/1 [00:03<00:00,  3.38s/it]


為使聽障者享有更無礙的聲音感受，國立陽明交通大學、振興醫院與國家實驗研究院國家儀器科技研究中心（國研院國儀中心）合作，成功開發出「微型法布里–珀羅光纖光學麥克風」（Miniaturized Fabry-Perot fiber-optic microphone based on capillary tube and hydrogel diaphragm），解決麥克風受磁場干擾之問題，其研究成果榮登光學領域頂級期刊《Optics & Laser Technology》。此新型微米級光纖麥克風結構簡單、成本低、訊號穩定、尺寸微小，整個麥克風的尺寸如同一根頭髮，而靈敏度比現有技術提升了約37%，能更細微地捕捉聲音變化，同時可以偵測更高頻的聲音，並且兼顧了輕薄微小的功能，未來可應用於穿戴式裝置上。
麥克風是一種將聲音轉換成電子訊號的換能器，在生物醫學影像、語音互動系統和助聽器等應用中發揮著至關重要的作用。然而一般麥克風容易受到強電磁場或是射頻干擾，也有電噪聲過大、靈敏度較低的問題。此次發表之新型微米級光纖麥克風，是利用光纖與光學元件來檢測聲波引起的變化，並將這些變化轉換為可測量的光信號。當聲音壓力作用在麥克風的膜片上，造成膜片的變形或振動，改變了光的干涉條紋，藉由此變化轉換為電子信號來偵測聲音。國立陽明交通大學生物醫學工程學系劉承揚教授、振興醫院耳鼻喉部力博宏主任醫師與國研院國儀中心協力合作，首先由力博宏主任醫師發想光纖麥克風的構造概念，接著由國儀中心協助薄膜（麥克風的膜片）製程與檢測，並由劉承揚教授團隊將光纖與薄膜結合，再進行測試與臨床驗證，最終成功開發出新型微米級光纖麥克風。整個麥克風的尺寸如同一根頭髮，同時具有優異的聲音感測靈敏度和穩定性。新型微米級光纖麥克風由於不含金屬材料，因此不受電磁干擾，適用於助聽器和人工電子耳，使用者可以自由進入強電磁場環境而不必擔心噪音的產生，在光聲成像、健康監測、無損檢測、醫學臨床等應用方面亦有具有巨大的商業潛力。
國研院國儀中心薄膜製程開發技術之前亦曾協助劉承揚教授團隊成功用蜘蛛絲製成光纖感測器，精準量測糖尿病患者血糖，近期更往高性能半導體材料應用發展。此次協助光纖薄膜製作，再次證明國儀中心是學術界挑戰世界頂尖科技的關鍵夥伴。國儀中心也期待與國內更多學者合作，應用頂尖的薄膜製程開發技術，研發出更多尖端生醫光電醫療器材。
[('麥克風', 

### 未用

In [ ]:
# https://www.nltk.org/
import nltk
import json
from google.colab import files # 在 Google Colab 環境上傳檔案時所用；若在個人電腦執行 Python 則不需要

nltk.download('punkt')

In [ ]:
# 若下載 python 檔並在自己電腦執行，開檔讀檔請用這段 # 若要上傳檔案到 Google Colab 虛擬主機，請註解掉這段
#f = open("D:/cjlin/sample/ReutersCorn-sample10.json", "r", encoding='UTF-8') # encoding='UTF-8' 才能正確讀入中文檔案
#docText = f.read()
#f.close()

In [ ]:
# 由電腦上傳檔案至 Colab 虛擬主機 # 若要在個人電腦單機作業，請註解掉這段
file = files.upload()
trf = open("ReutersCorn-train.json", "r", encoding='UTF-8')
fdoc = trf.read()
trf.close()

In [ ]:
dataset = json.loads(fdoc)
dataset[0]['text'] # 此行僅用來確認執行結果正確，可刪去

In [ ]:
file = files.upload()
dictf = open("posDict_ReutersCorn.json", "r", encoding='UTF-8')
dict_raw = dictf.read()
dictf.close()
POSdict = json.loads(dict_raw)

In [ ]:
wordTotal = 0
tokenSet = set()
for doc in dataset:
  word = nltk.word_tokenize(doc['text'])
  wordTotal += len(word)
  #nltk_token = []
  for w in word:
    tokenSet.add(w)
    #nltk_token.append(w)
  #doc['nltk_token'] = nltk_token
print(wordTotal)
len(tokenSet)

In [ ]:
lowerSet = set()
for w in tokenSet:
  lowerSet.add(w.lower())
len(lowerSet)

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

In [ ]:
nltk.download('omw-1.4')

In [ ]:
word_sets = ['automate', 'automatic', 'automation', 'computer', 'computerize', 'computerization', 'computerizational']
for w in word_sets:
  stemstr = lemmatizer.lemmatize(w, "v")
  print(w + ' ==> ' + stemstr)

In [ ]:
lemmaWOpos = set()
for w in lowerSet:
  lemmaWOpos.add(lemmatizer.lemmatize(w))
len(lemmaWOpos)

In [ ]:
lemmaWpos = set()
for w in lowerSet:
  pos = POSdict.get(w)
  if pos != None and pos != "x":
    if pos == "j": pos = "a"
    lemmaWpos.add(lemmatizer.lemmatize(w, pos))
  else:
    lemmaWpos.add(lemmatizer.lemmatize(w))
len(lemmaWpos)

In [ ]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [ ]:
stemSet = set()
for w in lowerSet:
  stemSet.add(stemmer.stem(w))
len(stemSet)

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [ ]:
word_sets = ['automate', 'automatic', 'automation', 'computer', 'computerize', 'computerization', 'computerizational']
for w in word_sets:
  stemstr = stemmer.stem(w)
  print(w + ' ==> ' + stemstr)


In [ ]:
pip install -U ckiptagger[tf,gdown]

In [ ]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [ ]:
data_utils.download_data_gdown("./")

In [ ]:
ws = WS("./data")
pos = POS("./data")

In [ ]:
sentence_list = [
    "土地公有政策?？還是土地婆有政策。.",
    "某某候選人提出的土地公有政策",
    "最多容納59,000個人,或5.9萬人,再多就不行了.這是環評的結論.",
    "電子計算機是會計算題目的機器。",
]


In [ ]:
sent_seg = ws(["高高興興打打球,丟丟看成績如何、開不開心"])

In [ ]:
sent_seg[0]

In [ ]:
sent_pos = pos(sent_seg)
sent_pos[0]